In [1]:
from math import *
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

## Partie Algorithmie - argument x

### Création du vecteur de départ

page numéroté num 156

In [2]:
def alp(p):
    i = np.arange(p+1)
    return (1/2) * (1 - np.cos( (2*i+1)*np.pi / (2*p+1)))

In [3]:
def bet(p):
    i = np.arange(p+1)
    return (1/2) * (1 - np.cos( 2*i*np.pi / (2*p+1)))

page numéroté num 157

In [4]:
def start_point(p):
    I = np.zeros((2*(p+1),1))
    I[:p+1,0] = alp(p)
    I[p+1:,0] = bet(p)
    return I

### Création de la Jacobienne

page numéroté num 157 puis 163

In [5]:
def a_prime_beta(p) : 
    #i verif
    i = np.arange(0,p) + 1
    #eta verif
    eta = ((2*(p-i)+1)*np.pi) / (2*p+1)
    
    #left et right verif
    left = (2*p*np.cos(p*eta)) / (np.cos(eta) + 1)
    right = (np.sin(p*eta) / np.sin(eta)) * ( 2*p + 2/(np.cos(eta) + 1))
    return left + right

In [6]:
def b_prime_alpha(p) :
    #i et teta vérif
    i = np.arange(0,p)
    teta = ( 2*(p-i)*np.pi) / (2*p+1)
    
    #left et right vérif
    left = (2*p*np.cos(p*teta)) / (np.cos(teta) - 1)
    right = (np.sin(p*teta) / np.sin(teta)) * ( 2*p - 2/(np.cos(teta) - 1))
    return left + right

In [7]:
def Jacobian_X0(p,h, x,function,eps) : 
    f = function(x,h)
    f_max = np.max(f)
    f_eps = max(f_max, eps)
    I = np.zeros(2*p)

    I[:p] = b_prime_alpha(p)
    I[p:] = a_prime_beta(p)
  
    return np.sqrt(f_eps)*np.diag(I)

### Création de l'interpolation a et b 

page numéroté num 155

In [8]:
def a(x,alpha,h, p,function, eps):
    res = 0
    for i in range(p+1):
        som = (-1)**(i+p)* np.sqrt(max(function(alpha[i],h),eps) / alpha[i])
        mul = 1 
        for j in range(p+1) :
            if i != j :
                mul = mul * ((x - alpha[j]) / (alpha[i] - alpha[j])) 
        som = som*mul
        res += som
    return res

In [9]:
def b(x,beta,h,p, function,eps):
    res = 0
    for i in range(p+1) : 
        som = (-1)**(i+p) * sqrt(max(function(beta[i],h), eps) / ( 1 - beta[i]))
        mul = 1 
        for j in range(p+1) : 
            if i != j :
                mul = mul * ((x - beta[j]) / (beta[i] - beta[j]))
        som = som*mul
        res += som
    return res

### Vecteur têta

page numéroté num 155

In [10]:
def teta_teta(x, h, p, function,eps) : 
    teta = np.zeros((2*p,1))
    alpha = x[:p]
    alpha_new = np.concatenate(( alpha, [[1]]))
    beta = x[p:]
    beta_new = np.concatenate(([[0]],beta))
    
    teta[:p] = b(alpha, beta_new, h, p, function,eps)
    teta[p:] = a(beta, alpha_new, h, p, function, eps) 
    return teta 

### Splitting node 

### Algorithme Newton-Raphson

page numéroté num 155

In [11]:
def G_hat(x,h, p, function,eps) : 
    J = Jacobian_X0(p,h,x, function, eps)
    tet = teta_teta(x, h, p, function,eps)
    d = npl.solve(J,tet)
    return x - d

In [12]:
def Newton_Raphson(p,h ,function,eps=1e-12)  :
    X0 = start_point(p)
    X = np.concatenate((X0[:p], X0[p+2:]))
    X_past = X + 2*eps
    err = 2*eps
    ERR = []
    i = 0
    while npl.norm(X-X_past)/npl.norm(X) > eps and i<20000:
        X_past = X
        X = G_hat(X,h,p,function,eps)
        i+=1
        ERR += [npl.norm(X - X_past)/npl.norm(X)]
    print("Nombre d'itérations finales : ",i)
    #print("Erreur Newton-Raphson : ", ERR[-1])
    return X, ERR, i

### Création des points d'interpolation = Lukacs

In [13]:
def polynom(x, a, b):
    sol = x*(a**2) +(1-x)*(b**2)
    return sol

### Affichage du polynome

In [14]:
def polynom_display(xn,poln, x, pol,h, real_func):
    x0 = np.linspace(0,1,100)
    
    plt.figure()
    plt.plot(x, pol, '-b')
    plt.plot(xn,poln, 'ob', label ="Points d'interpolation")
    plt.plot(x0, real_func(x0,h), '-r', label = "Fonction à interpoler")
    plt.legend()
    plt.show()   

In [15]:
def polynom_display2(xn,pol,h, real_func):
    x0 = np.linspace(0,1,100)
    
    plt.plot(xn,pol, 'ob', label ="Points d'interpolation")
    plt.plot(x0, real_func(x0,h), '-r', label = "Fonction à interpoler")
    plt.title("Interpolation de la fonction")

### Interpolation par un polynome

In [16]:
def interpol(p,h,function, eps=1e-3) :
    X, ERR, nit = Newton_Raphson(p, h, function, eps)
    X = np.concatenate((X[:p], [[1]], [[0]], X[p:]))
    
    xn = np.zeros((p+1,1))
    xn[:,0] = np.linspace(0,1,p+1)
    
    ar = X[:p+1]
    br = X[p+1:]
    
    A = a(xn, ar, h, p, function,eps )
    B = b(xn, br, h, p, function,eps )
    poln = polynom(xn, A, B)

    x = np.linspace(0,1,300)
    A_pol = a(x, ar, h, p ,function, eps)
    B_pol = b(x, br, h, p ,function, eps)
    pol = polynom(x, A_pol, B_pol)

    
    return poln, xn, ERR, pol, x, nit 

In [17]:
def erreur(xn, poln, function, h) : 
    f = function(xn, h)
    dif = np.abs(f - poln)
    n = np.shape(dif)[0]
    dif = np.reshape(dif, (n))
    xn = np.reshape(xn, (n))
    som = 0
    for i in range(n-1) : 
        som += (xn[i+1]-xn[i]) * (dif[i+1]+dif[i]) / 2
    return som 

In [18]:
def find_order(x, err) :
    return (log10(err[-1])-log10(err[0])) /  (log10(x[-1])-log10(x[0]))